# Tracking Trump: Electoral Votes

[Morning Consult](https://morningconsult.com) has a page called [Tracking Trump](https://morningconsult.com/tracking-trump/) that summarizes the presidential approval polls on a state-by-state basis, and tells you how many states Trump has a net positive or net negative approval rating. But I don't care about the number of *states*; I care about the number of *electoral votes*. This notebook computes that for me.

In [1]:
import urllib.request
import re
from collections import namedtuple, defaultdict

Row = namedtuple('Row', 'state delta jan17app jan17dis jan17err app dis err')

def read_data(url='https://morningconsult.com/tracking-trump/'):
    "Fetch data from the website and return a list of `Row` objects."
    with urllib.request.urlopen(url) as response:
        html = response.read().decode('utf-8')
        rows = re.findall(r'<tr(.*?)</tr>', html, re.S)
        return [parse_row(row) for row in rows[1:]]
    
def parse_row(row):
    "Parse an html string into a `Row` object."
    state, *nums =  re.findall('>([^>]*?)</td', row.replace('%', ''))
    return Row(state, *map(int, nums))

def net(row): "Approval minus disapproval."; return row.app - row.dis 

rows = read_data()

def EV(swing=0, rows=rows):
    "How many electoral votes would Trump win today, given a swing."
    return sum(votes[row.state] for row in rows if net(row) + swing > 0)

votes = dict(AL=9,  AK=3,  AZ=11, AR=6,  CA=55, CO=9,  CT=7,  DE=3,  DC=3,  FL=29, 
             GA=16, HI=4,  ID=4,  IL=20, IN=11, IA=6,  KS=6,  KY=8,  LA=8,  ME=4, 
             MD=10, MA=11, MI=16, MN=10, MS=6,  MO=10, MT=3,  NE=5,  NV=6,  NH=4,  
             NJ=14, NM=5,  NY=29, NC=15, ND=3,  OH=18, OK=7,  OR=7,  PA=20, RI=4,  
             SC=9,  SD=3,  TN=11, TX=38, UT=6,  VT=3,  VA=13, WA=12, WV=5,  WI=10, WY=3)

In [2]:
EV()

164

This says that Trump has a net positive approval (i.e. more approval than disapproval) in states with a total of **164** electoral votes. You need **270** to win.

But of course these are approval polls, not ballots, and don't translate directly to votes. Things can change; the election is a long ways away, we don't know who's running, we don't know if there are third party candidate(s), and we don't know if there is systematic bias in the polling data. In the table below, I list the number of electoral votes Trump would get assuming he gets an increase of net approval of 0 to 9 percentage points across the board in every state. We see he would need a **7** percent upswing from the polling data in order to win.

In [3]:
{m: EV(m) for m in range(10)}

{0: 164,
 1: 164,
 2: 164,
 3: 164,
 4: 255,
 5: 255,
 6: 255,
 7: 279,
 8: 279,
 9: 294}

Here are the states sorted by current net approval:

In [4]:
def undecided(row): return 100 - row.app - row.dis

for r in sorted(rows, key=net):
    print('{}: {:2d} EV, net {:+3d} (+{} -{} ?{})'
           .format(r.state, votes[r.state], net(r), r.app, r.dis, undecided(r)))

DC:  3 EV, net -62 (+17 -79 ?4)
VT:  3 EV, net -31 (+33 -64 ?3)
CA: 55 EV, net -29 (+33 -62 ?5)
MA: 11 EV, net -29 (+33 -62 ?5)
HI:  4 EV, net -28 (+34 -62 ?4)
MD: 10 EV, net -28 (+34 -62 ?4)
NY: 29 EV, net -23 (+36 -59 ?5)
WA: 12 EV, net -23 (+36 -59 ?5)
OR:  7 EV, net -22 (+37 -59 ?4)
IL: 20 EV, net -21 (+37 -58 ?5)
CT:  7 EV, net -20 (+38 -58 ?4)
NM:  5 EV, net -19 (+38 -57 ?5)
RI:  4 EV, net -19 (+38 -57 ?5)
NJ: 14 EV, net -18 (+39 -57 ?4)
CO:  9 EV, net -16 (+40 -56 ?4)
DE:  3 EV, net -16 (+40 -56 ?4)
MN: 10 EV, net -14 (+41 -55 ?4)
MI: 16 EV, net -12 (+41 -53 ?6)
WI: 10 EV, net -12 (+42 -54 ?4)
NV:  6 EV, net -11 (+42 -53 ?5)
IA:  6 EV, net -10 (+43 -53 ?4)
AZ: 11 EV, net  -8 (+44 -52 ?4)
NH:  4 EV, net  -8 (+44 -52 ?4)
ME:  4 EV, net  -6 (+45 -51 ?4)
PA: 20 EV, net  -6 (+45 -51 ?4)
FL: 29 EV, net  -3 (+46 -49 ?5)
GA: 16 EV, net  -3 (+46 -49 ?5)
NC: 15 EV, net  -3 (+46 -49 ?5)
OH: 18 EV, net  -3 (+46 -49 ?5)
VA: 13 EV, net  -3 (+46 -49 ?5)
KS:  6 EV, net  +2 (+49 -47 ?4)
MO: 10 E

Here are all the states with more than 5% undecided. This is evidence that most people have made up their mind.

In [5]:
{r.state: undecided(r)
 for r in rows if undecided(r) > 5}

{'AK': 6, 'MI': 6, 'TX': 6}